# Examples for a basic query against OEDI for ResStock

## Import modules

In [1]:
from buildstock_query import BuildStockQuery
import pandas as pd
import toml

## Reload changes
If you are a developer and are making changes to the code, signal to reload changes each time a block is executed

In [2]:
# auto reload
%load_ext autoreload
%autoreload 2


## Initialize BuildStock Query Object
Connect to the database, setup the tables, set the source schema, and set project workgroup

In [3]:
# specify your baseline, upgrade and timeseries tables separately
my_run = BuildStockQuery(
    'rescore', # Change to your project working group
    'buildstock_sdr',
    (
        "resstock_2024_amy2018_release_2_metadata_state_vu", # Baseline metadata view
        "resstock_2024_amy2018_release_2_by_state_vu", # Timeseries view
        "resstock_2024_amy2018_release_2_metadata_state_vu" # Upgrade metadata view (same as Baseline for OEDI)
    ),
    db_schema="resstock_oedi",
    skip_reports=True
)

INFO:buildstock_query.query_core:Loading ('resstock_2024_amy2018_release_2_metadata_state_vu', 'resstock_2024_amy2018_release_2_by_state_vu', 'resstock_2024_amy2018_release_2_metadata_state_vu') ...


INFO:botocore.tokens:Loading cached SSO token for nrel-sso


In [3]:
# or pass a db-schema path that has it all
my_db_schema = toml.load("resstock_oedi_vu.toml")
my_run = BuildStockQuery(
    'rescore', # Change to your project working group
    db_name='buildstock_sdr',
    table_name="resstock_2024_amy2018_release_2",
    db_schema=my_db_schema,
    skip_reports=True
)

INFO:buildstock_query.query_core:Loading resstock_2024_amy2018_release_2 ...


INFO:botocore.tokens:Loading cached SSO token for nrel-sso


## Simple annual query

In [6]:
baseline_agg = my_run.query(
    enduses=['electricity.total.energy_consumption'],
    group_by=['state', 'geometry_building_type_recs'],
)
baseline_agg

INFO:botocore.tokens:Loading cached SSO token for nrel-sso
INFO:botocore.tokens:Loading cached SSO token for nrel-sso
/Users/radhikar/Documents/buildstock2025/buildstock-query-applied_only/buildstock_query/report_query.py:95: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  change_df = change_df.fillna(0)


,state,geometry_building_type_recs,sample_count,units_count,electricity.total.energy_consumption
0,AL,Mobile Home,1233,3.110879e+05,3.841222e+09
1,AL,Multi-Family with 2 - 4 Units,450,1.135357e+05,1.098778e+09
2,AL,Multi-Family with 5+ Units,1071,2.702151e+05,2.040843e+09
3,AL,Single-Family Attached,143,3.607913e+04,4.530630e+08
4,AL,Single-Family Detached,6223,1.570073e+06,2.567576e+10
...,...,...,...,...,...
239,WY,Mobile Home,150,3.784525e+04,2.784647e+08
240,WY,Multi-Family with 2 - 4 Units,74,1.867032e+04,1.422061e+08
241,WY,Multi-Family with 5+ Units,103,2.598707e+04,1.829168e+08
242,WY,Single-Family Attached,53,1.337199e+04,1.306436e+08


## Simple timeseries query

In [7]:
ts_agg = my_run.query(
    enduses=['out.electricity.total.energy_consumption'],
    annual_only=False,
    restrict=[('state', ['CO'])],
    upgrade_id=0,
    timestamp_grouping_func='month',
    group_by=['geometry_building_type_recs', 'state'],
    get_query_only=False
)
print(ts_agg)

/Users/radhikar/Documents/buildstock2025/buildstock-query-applied_only/buildstock_query/report_query.py:95: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  change_df = change_df.fillna(0)


    timestamp    geometry_building_type_recs state  sample_count  \
0  2018-01-01                    Mobile Home    CO           391   
1  2018-01-01  Multi-Family with 2 - 4 Units    CO           469   
2  2018-01-01     Multi-Family with 5+ Units    CO          2001   
3  2018-01-01         Single-Family Attached    CO           664   
4  2018-01-01         Single-Family Detached    CO          5900   
5  2018-02-01                    Mobile Home    CO           391   
6  2018-02-01  Multi-Family with 2 - 4 Units    CO           469   
7  2018-02-01     Multi-Family with 5+ Units    CO          2001   
8  2018-02-01         Single-Family Attached    CO           664   
9  2018-02-01         Single-Family Detached    CO          5900   
10 2018-03-01                    Mobile Home    CO           391   
11 2018-03-01  Multi-Family with 2 - 4 Units    CO           469   
12 2018-03-01     Multi-Family with 5+ Units    CO          2001   
13 2018-03-01         Single-Family Attached    

## Utility unit count query

In [6]:
units = my_run.utility.aggregate_unit_counts_by_eiaid(
    eiaid_list=['4110', '14328'], # ComEd and PG&E,
    get_query_only=False,
    group_by=['state']
)
print(units)
units.to_csv("utility_units.csv")

INFO:buildstock_query.utility_query:Aggregating unit counts by eiaid
INFO:botocore.tokens:Loading cached SSO token for nrel-sso
INFO:botocore.tokens:Loading cached SSO token for nrel-sso


   eiaid state  sample_count   units_count
0  14328    CA         28730  1.828840e+09
1   4110    IL         15880  1.010859e+09


## Utility annual query

In [10]:
enduses = [
    'out.site_energy.net.energy_consumption',
    'out.site_energy.total.energy_consumption'
]

# By default just does all the eiaids
ts_utility_agg = my_run.utility.aggregate_annual_by_eiaid(
    enduses=enduses,
    group_by=['state'],
    get_query_only=False
)
print(ts_utility_agg)
ts_utility_agg.to_csv("utility_annual_agg.csv")

      eiaid state  sample_count   units_count  \
0     10000    KS          1774  1.129260e+08   
1     10000    MO          2453  1.561485e+08   
2     10005    KS          1801  1.146447e+08   
3     10009    TX          3206  2.040815e+08   
4     10012    OK           319  2.030630e+07   
...     ...   ...           ...           ...   
1901   9964    KY           658  4.188572e+07   
1902    998    KS           108  6.874861e+06   
1903   9991    MN           385  2.450761e+07   
1904   9996    KS           278  1.769640e+07   
1905   9999    IN          1976  1.257845e+08   

      site_energy.net.energy_consumption  site_energy.total.energy_consumption  
0                           4.152121e+12                          4.152489e+12  
1                           5.237433e+12                          5.242919e+12  
2                           3.853915e+12                          3.853915e+12  
3                           3.782175e+12                          3.807804e+12  
4     

## Utility timeseries query

In [11]:
enduses = [
    'out.site_energy.net.energy_consumption',
    'out.site_energy.total.energy_consumption'
]
group_by=['state', 'timestamp'] # If no timestamp, then annual values will be returned

ts_utility_agg = my_run.utility.aggregate_ts_by_eiaid(
    eiaid_list=['4110', '14328'], # ComEd and PG&E
    enduses=enduses,
    group_by=group_by,
    get_query_only=False
)
print(ts_utility_agg)
ts_utility_agg.to_csv("utility_ts_agg.csv")

INFO:buildstock_query.utility_query:Will submit request for ['4110', '14328']
INFO:buildstock_query.utility_query:Submitting query for ['4110', '14328']
/Users/radhikar/Documents/buildstock2025/buildstock-query-applied_only/buildstock_query/report_query.py:95: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  change_df = change_df.fillna(0)
INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 0.
INFO:buildstock_query.query_core:{'submitted': 0, 'running': 0, 'pending': 1, 'completed': 0, 'failed': 0}
INFO:buildstock_query.query_core:Submitted queries[0] (40a386d6-2d81-4332-9b50-b836c57e5f34)
INFO:buildstock_query.query_core:{'submitted': 1, 'running': 0, 'pending': 1, 'completed': 0, 'failed': 0}
INFO:buildstock_query.query_core:{'submitted': 1, 'running': 1, 

       eiaid state           timestamp  sample_count   units_count  \
0       4110    IL 2018-06-16 05:15:00         15880  2.563967e+06   
1       4110    IL 2018-06-16 05:30:00         15880  2.563967e+06   
2       4110    IL 2018-06-16 05:45:00         15880  2.563967e+06   
3       4110    IL 2018-06-16 06:00:00         15880  2.563967e+06   
4       4110    IL 2018-06-16 06:15:00         15880  2.563967e+06   
...      ...   ...                 ...           ...           ...   
70075  14328    CA 2018-06-26 03:15:00         28730  3.417140e+06   
70076  14328    CA 2018-06-26 03:30:00         28730  3.417140e+06   
70077  14328    CA 2018-06-26 03:45:00         28730  3.417140e+06   
70078  14328    CA 2018-06-26 04:00:00         28730  3.417140e+06   
70079  14328    CA 2018-06-26 04:15:00         28730  3.417140e+06   

       site_energy.net.energy_consumption  \
0                           580328.455098   
1                           582542.029626   
2                       

In [12]:
my_run.save_cache()

INFO:buildstock_query.query_core:10 queries cache saved to .bsq_cache/resstock_2024_amy2018_release_2_query_cache.pkl
